<a href="https://colab.research.google.com/github/VIPlearner/csv-files/blob/main/Hopefully%20greater.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from sklearn.linear_model import Ridge, Lasso
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, MaxAbsScaler, MinMaxScaler, Normalizer, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import VarianceThreshold
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.metrics import mean_squared_error
from sklearn.tree import DecisionTreeRegressor

In [ ]:
df=pd.read_csv("https://raw.githubusercontent.com/VIPlearner/csv-files/main/Train.csv", index_col="VehicleID")
#dropping irrelevant features
#df.drop(['Colour'], axis = 1, inplace = True)
#df.drop(['Maker'], axis = 1, inplace = True)
#df.drop(['Model'], axis = 1, inplace = True)
df['Year'] = df['Year'].str.replace(',', '').astype(float)
df['Distance'] = df['Distance'].str.replace(',', '').astype(float)
df['Location'] = df.Location.astype('category')
df['Type'] = df.Type.astype('category')
df['Maker'] = df.Maker.astype('category')
df['Model'] = df.Model.astype('category')
df['Colour'] = df.Colour.astype('category')
df = df[~(np.isnan(df["Amount (Million Naira)"]))]
X_train, X_test, y_train, y_test = train_test_split(df.drop(['Amount (Million Naira)'], axis = 1), df['Amount (Million Naira)'],
                                                    test_size = 0.3,
                                                   random_state = 42)
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7188 entries, VHL12546 to VHL15246
Data columns (total 8 columns):
 #   Column                  Non-Null Count  Dtype   
---  ------                  --------------  -----   
 0   Location                7188 non-null   category
 1   Maker                   7188 non-null   category
 2   Model                   7188 non-null   category
 3   Year                    7167 non-null   float64 
 4   Colour                  7188 non-null   category
 5   Amount (Million Naira)  7188 non-null   float64 
 6   Type                    6992 non-null   category
 7   Distance                4836 non-null   float64 
dtypes: category(5), float64(3)
memory usage: 570.1+ KB


In [ ]:
#numeric_features = ["Year", "Type"]
year_transformer = Pipeline(
    steps=[("imputer", SimpleImputer(strategy="most_frequent")), ("scaler", MinMaxScaler())]
)

distance_transformer = Pipeline(
    steps=[("imputer", SimpleImputer(strategy="mean")), ("scaler", MinMaxScaler())]
)

categorical_features = ["Location", "Type", 'Maker', 'Model', 'Colour']
categorical_transformer = OneHotEncoder(handle_unknown="ignore")

preprocessor = ColumnTransformer(
    transformers=[
        ("year", year_transformer, ["Year"]),
        ("distance", year_transformer, ["Distance"]),
        ("cat", categorical_transformer, categorical_features),
    ]
)



In [ ]:
pipeline = Pipeline(
  steps=[("preprocessor", preprocessor), ("estimator", DecisionTreeRegressor())]
)



In [ ]:
#pipeline.get_params().keys()

In [ ]:
parameters = {#'estimator__alpha': [1],
              #'estimator':[DecisionTreeRegressor()],
              'estimator__criterion' : ['gini', 'entropy'],
              'estimator__max_depth' : [2,4,6,8,10,12]
}

In [ ]:
grid = GridSearchCV(pipeline, parameters, cv=5)

grid.fit(X_train, y_train)


/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:372: FitFailedWarning: 
60 fits failed out of a total of 60.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
30 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.7/dist-packages/sklearn/pipeline.py", line 394, in fit
    self._final_estimator.fit(Xt, y, **fit_params_last_step)
  File "/usr/local/lib/python3.7/dist-packages/sklearn/tree/_classes.py", line 1320, in fit
    X_idx_sorted=X_idx_sorted,
  File "/usr/local/lib/python3.7/dist-packages/s

KeyError: ignored

In [ ]:
grid.best_params_

{'estimator': Ridge(alpha=1),
 'estimator__alpha': 1,
 'preprocessor__distance__imputer__strategy': 'mean',
 'preprocessor__distance__scaler': MinMaxScaler(),
 'preprocessor__year__imputer__strategy': 'most_frequent',
 'preprocessor__year__scaler': MinMaxScaler()}

In [ ]:
print("model score: %.3f" % grid.score(X_test, y_test))

model score: 0.701


In [ ]:
y_pred = grid.predict(X_test)
errors = mean_squared_error(y_test, y_pred, squared = False)
print(errors)

11.958614301967241


In [ ]:
test_df=pd.read_csv("https://raw.githubusercontent.com/VIPlearner/csv-files/main/Test.csv")
test_df.head()
#dropping irrelevant features
# test_df.drop(['Colour'], axis = 1, inplace = True)
# test_df.drop(['Maker'], axis = 1, inplace = True)
# test_df.drop(['Model'], axis = 1, inplace = True)
test_df['Year'] = test_df['Year'].str.replace(',', '').astype(float)
#test_df['Distance'] = test_df['Distance'].str.replace(',', '').astype(float)
test_df['Location'] = test_df.Location.astype('category')
test_df['Type'] = test_df.Type.astype('category')
test_df['Maker'] = test_df.Maker.astype('category')
test_df['Model'] = test_df.Model.astype('category')
test_df['Colour'] = test_df.Colour.astype('category')

In [ ]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2061 entries, 0 to 2060
Data columns (total 8 columns):
 #   Column     Non-Null Count  Dtype   
---  ------     --------------  -----   
 0   VehicleID  2061 non-null   object  
 1   Location   2061 non-null   category
 2   Maker      2061 non-null   category
 3   Model      2061 non-null   category
 4   Year       2059 non-null   float64 
 5   Colour     2061 non-null   category
 6   Type       2007 non-null   category
 7   Distance   1385 non-null   float64 
dtypes: category(5), float64(2), object(1)
memory usage: 83.6+ KB


In [ ]:
test_y_pred = grid.predict(test_df)

In [ ]:
test_pred_df = pd.DataFrame({'VehicleID' : test_df['VehicleID'], 'Amount (Million Naira)': test_y_pred})
test_pred_df.head

<bound method NDFrame.head of      VehicleID  Amount (Million Naira)
0     VHL18518               15.637752
1     VHL17149                8.538268
2     VHL10927                3.488541
3     VHL12909                4.801283
4     VHL12348                9.135657
...        ...                     ...
2056  VHL17903               15.467421
2057  VHL14018               11.909886
2058  VHL17473               10.896719
2059  VHL11480               13.374750
2060  VHL13881                5.872364

[2061 rows x 2 columns]>

In [ ]:
test_pred_df.to_csv('thirdtry.csv', encoding='utf-8', index=False)

In [ ]:
from google.colab import files
files.download("thirdtry.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import tensorflow as tf
tf.test.gpu_device_name()

'/device:GPU:0'

In [ ]:
help(DecisionTreeRegressor)

Help on class DecisionTreeRegressor in module sklearn.tree._classes:

class DecisionTreeRegressor(sklearn.base.RegressorMixin, BaseDecisionTree)
 |  DecisionTreeRegressor(*, criterion='squared_error', splitter='best', max_depth=None, min_samples_split=2, min_samples_leaf=1, min_weight_fraction_leaf=0.0, max_features=None, random_state=None, max_leaf_nodes=None, min_impurity_decrease=0.0, ccp_alpha=0.0)
 |  
 |  A decision tree regressor.
 |  
 |  Read more in the :ref:`User Guide <tree>`.
 |  
 |  Parameters
 |  ----------
 |  criterion : {"squared_error", "friedman_mse", "absolute_error",             "poisson"}, default="squared_error"
 |      The function to measure the quality of a split. Supported criteria
 |      are "squared_error" for the mean squared error, which is equal to
 |      variance reduction as feature selection criterion and minimizes the L2
 |      loss using the mean of each terminal node, "friedman_mse", which uses
 |      mean squared error with Friedman's improv